In [440]:
# Import librarys
import pandas as pd
import numpy as np

In [441]:
# Define function to read in excel file
def readfile(year, ext, sheet):
    
        df = pd.read_excel('./supplementary/' + year + ext, 
                                sheet_name=sheet,
                                    header=None)
        return df
    
# Define tidy up initial df
def tidy(df):
    
    #### Fix usual issues with all strings
    
    # Capitalise columns
    df = df.applymap(lambda x: x.upper() if type(x) is str else x)

    # Strip whitespace
    df = df.applymap(lambda x: x.strip() if type(x) is str else x)

    # Remove parenthesis
    df = df.applymap(lambda x: x.replace('(', '') if type(x) is str else x)
    df = df.applymap(lambda x: x.replace(')', '') if type(x) is str else x)

    # Replace annoying substrings
    df = df.applymap(lambda x: x.replace(' AND ', ' & ') if type(x) is str else x)
    df = df.applymap(lambda x: x.replace(' – ', ' - ') if type(x) is str else x)
    df = df.applymap(lambda x: x.replace(' / ', '/') if type(x) is str else x)
    df = df.applymap(lambda x: x.replace('/ ', '/') if type(x) is str else x)
    df = df.applymap(lambda x: x.replace(' /', '/') if type(x) is str else x)
    
    #### Drop rows and columns with a lot of empty values
    
    # Drop rows with minimum number of 3 non-null values
    df = df.dropna(thresh=3)

    # Drop rows with columns number of 3 non-null values
    df = df.dropna(axis='columns', thresh=3)
    
    return df

In [442]:
# Create empty list to store raw df
rframes = []

# Loop through years
years = ['2012', '2017', '2018', '2019', '2020', '2021', '2022']

for y in years:

    # Read in sheet
    wdf = readfile(y, '.xlsx', 'Table 3.7')

    wdf = tidy(wdf)

    # Append to list of df (with year)
    rframes.append([wdf, y])
    
# Loop through years
years = ['2011', '2013', '2014', '2015', '2016']

for y in years:

    # Read in sheet
    wdf = readfile(y, '.xls', 'Table 3.7')

    wdf = tidy(wdf)

    # Append to list of df (with year)
    rframes.append([wdf, y])
    
# Loop through years
years = ['2010']

for y in years:

    # Read in sheet
    wdf = readfile(y, '.xls', '3.7')

    wdf = tidy(wdf)

    # Append to list of df (with year)
    rframes.append([wdf, y])

In [443]:
#wdf.head()

In [444]:
#### Pull out subtables from sheet

# Create a list for tables
tables_ls = []

# Create list of subtables on sheet
ls = ['REASON FOR SUPPORT']

# Loop through frames
for f in rframes:
    
    # Select df which is first element of list entry
    df = f[0]

    # Not sure what this does (store the recurrence of 'markers')
    groups = df[0].isin(ls).cumsum()

    # Not sure how this works, creates a dictionary of df for each table with key TABLE1, TABLE2, ...
    tables = {'TABLE'+str(k): g.iloc[0:] for k,g in df.groupby(groups)}

    # Append to list of df with year
    tables_ls.append([tables, f[1]])

In [445]:
#### Tidy up column headers for each table

# Loop though list of tables
for l in tables_ls:
    
    # Select df which is first element of list entry
    tables = l[0]
    
    # Loop through tables dictionary
    for t in tables:
        
        if len(tables[t].columns) == 4:
        
            # Manually rename columns
            tables[t].columns = ['REASON FOR SUPPORT', 
                                     'FEMALE', 'MALE', 'TOTAL']
            
        if len(tables[t].columns) == 7:
           
            # Manually rename columns
            tables[t].columns = ['REASON FOR SUPPORT', 'FEMALE', 'MALE', 'TOTAL',
                                        'FEMALE per 1000', 'MALE per 1000', 'TOTAL per 1000']            

    # Drop 1st row
    tables[t] = tables[t].iloc[1:].reset_index(drop=True)

In [446]:
#tables_ls[0][0]
#read_df.head()
#tables['TABLE1']
#tables[t]

In [455]:
# Create empty list to store long format df
lframes = []

# Loop though list of tables
for l in tables_ls:
    
    # Select df which is first element of list entry
    tables = l[0]

    # Loop through tables
    for t in tables:        
    
        # Loop through headers in table
        for h in tables[t].columns[1:]:
            
            # Define working df
            wdf = tables[t]

            # Melt data into long format
            ldf = pd.melt(wdf, id_vars=[wdf.columns[0]], value_vars=[h],
                         var_name='Type', value_name='Value')

            # Set first column header to be Category
            ldf.columns.values[0] = 'Category'

            # Create Table column with value equal to title of table 
            ldf['Table'] = t

            # Create Year column with value from second element of list
            ldf['Year'] = l[1]

            # Append to list of df
            lframes.append(ldf)

In [461]:
ldf.head()

,Category,Type,Value,Table,Year
0,PUPILS FOR WHOM REASON FOR SUPPORT IS REPORTED 1,TOTAL per 1000,94.250491,TABLE0,2010
1,LEARNING DISABILITY,TOTAL per 1000,10.753545,TABLE0,2010
2,DYSLEXIA,TOTAL per 1000,18.726517,TABLE0,2010
3,OTHER SPECIFIC LEARNING DIFFICULTY E.G. NUMERIC,TOTAL per 1000,9.338343,TABLE0,2010
4,OTHER MODERATE LEARNING DIFFICULTY,TOTAL per 1000,11.972746,TABLE0,2010


In [466]:
## Concat all together long format dfs into final df
fdf = pd.concat(lframes)

In [467]:
## Final Tidy up

# Sort bad values
fdf['Value'] = fdf['Value'].replace('-', 0)

# Remove suffix
fdf['Category'] = fdf['Category'].replace('PUPILS FOR WHOM REASON FOR SUPPORT IS REPORTED 1',
                                              'PUPILS FOR WHOM REASON FOR SUPPORT IS REPORTED')

# Deal with second table for 2022

# Function to add 'per 1000' if Table value is TABLE2
def fix_category(row):
    
    if row['Table'] == 'TABLE2':
        return row['Type'] + ' per 1000'
    else:
        return row['Type']
        
# Apply function above to each row
fdf['Type'] = fdf.apply(lambda row: fix_category(row), axis=1)

# Drop Table column
fdf = fdf.drop('Table', axis=1)

In [468]:
fdf.head()

,Category,Type,Value,Year
0,PUPILS FOR WHOM REASON FOR SUPPORT IS REPORTED,FEMALE,18335,2012
1,LEARNING DISABILITY,FEMALE,1699,2012
2,DYSLEXIA,FEMALE,3165,2012
3,OTHER SPECIFIC LEARNING DIFFICULTY E.G. NUMERIC,FEMALE,1748,2012
4,OTHER MODERATE LEARNING DIFFICULTY,FEMALE,2323,2012


In [469]:
fdf.to_csv('supplementary.csv')